In [ ]:
### please specify your input path here
INPUT_TEST_FILE = "YOUR_TEST_FILE" # csv file list locations / paths to test cases (dicom)
OUTPUT_FOLDER = "YOUR_OUTPUT_DIR" # output directory

In [1]:
import os
import pandas as pd
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.listdir('/kaggle/input/mmdetection-2-17-offline')
#
# !pip install /kaggle/input/mmdetection-2-17-offline/mmcv_full-1.3.14-cp37-cp37m-linux_x86_64.whl --no-deps
# !pip install /kaggle/input/mmdetection-2-17-offline/pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl --no-deps
# !pip install /kaggle/input/mmdetection-2-17-offline/terminaltables-3.1.0-py3-none-any.whl --no-deps
# !pip install /kaggle/input/mmdetection-2-17-offline/pytest_runner-5.3.1-py3-none-any.whl --no-deps
# !pip install /kaggle/input/mmdetection-2-17-offline/mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl --no-deps
# !pip install /kaggle/input/mmdetection-2-17-offline/terminal-0.4.0-py3-none-any.whl --no-deps
# !pip install /kaggle/input/mmdetection-2-17-offline/mmdet-2.17.0-py3-none-any.whl --no-deps
# !pip install /kaggle/input/mmdetection-2-17-offline/addict-2.4.0-py3-none-any.whl --no-deps
# !pip install /kaggle/input/mmdetection-2-17-offline/yapf-0.31.0-py2.py3-none-any.whl --no-deps

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from model_cam import MDAIModel

pipeline = MDAIModel()

In [5]:
df = pd.read_csv(INPUT_TEST_FILE)
print(df.shape)

(102, 3)


In [6]:
import cv2
import numpy as np

output_folder = OUTPUT_FOLDER
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [7]:
import SimpleITK as sitk
import warnings

def read_from_DICOM_dir(DICOM_dir, dtype=sitk.sitkInt16):
    reader = sitk.ImageSeriesReader()
    list_series_ids = reader.GetGDCMSeriesIDs(DICOM_dir)

    sum_series = len(list_series_ids)
    if sum_series > 1:
        warnings.warn('Multiple series ids in this dir, only read one series')

    series_uid = list_series_ids[0]
    file_names = reader.GetGDCMSeriesFileNames(DICOM_dir, series_uid)
    image_nii = sitk.ReadImage(file_names, dtype)

    return image_nii

In [8]:
def cam_to_voxel_space(cam: np.ndarray, voxel:np.ndarray) -> np.ndarray:
    length, h, w = voxel.shape
    
    grayscale_cam_resized = np.zeros((length, h, w))
    cam_length = cam.shape[0]
    frame_interval = cam_length / length
    max_index = cam_length - 1
    
    offset_index = int((length - max_index * length / cam_length) / 2)
    
    for i in range(offset_index):
        grayscale_cam_resized[i] = cv2.resize(cam[0], (w, h)) * (0.5 + 0.5 * i / offset_index)
        grayscale_cam_resized[length - i - 1] = cv2.resize(cam[0], (w, h)) * (0.5 + 0.5 * i / offset_index)
        
    for j in range(offset_index, length - offset_index):
        # Find the indices of the original frames that we'll interpolate between
        ind = j - offset_index
        idx1 = int(ind * frame_interval)
        idx2 = min(idx1 + 1, max_index)

        # Find the interpolation weight
        alpha = (ind * frame_interval) - idx1

        # Interpolate between the original frames
        cam1 = cv2.resize(cam[idx1], (w, h))
        cam2 = cv2.resize(cam[idx2], (w, h))
        grayscale_cam_resized[j] = cv2.addWeighted(cam1, 1 - alpha, cam2, alpha, 0)

    return grayscale_cam_resized

In [ ]:
# iterate over rows of df for inference:
from tqdm import tqdm
output = []
idx = 0
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    # if idx < 36:
    #     idx += 1
    #     continue
    cube_id = row['StudyInstanceUID']
    cube_dir = row['image_folder']
    cam_ = pipeline.explain_gradcam({"scan_dir": cube_dir})
    img = sitk.GetArrayFromImage(read_from_DICOM_dir(cube_dir))
    cam_ = cam_to_voxel_space(cam_, img)
        
    # Define the file path for saving
    save_path = os.path.join(output_folder, f"{cube_id}.npy")

    # Save the resized_cam as a .npy file
    np.save(save_path, cam_)